In [1]:
#!unzip Test.zip

In [2]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
suffix = '.pgm'
kaggle_files = os.listdir('Test')
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [4]:
len(pgm_kaggle_files)

8000

In [5]:
kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = 'Test/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
  kaggle_patches.append(image)

  0%|          | 0/8000 [00:00<?, ?it/s]

100%|██████████| 8000/8000 [00:01<00:00, 5266.15it/s]


In [6]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

100%|██████████| 8000/8000 [00:00<00:00, 4497310.28it/s]


In [7]:
import joblib

In [8]:
# Aquí debe utilizar su modelo
model = joblib.load('models/knn_model.joblib')

In [9]:
from skimage import feature

# Aquí debe procesar las imagenes de pgm_kaggle_files
X_kag = []

suffix = '.pgm'
dir = os.listdir('Test/')
pgm_files = [filename for filename in dir if filename.endswith(suffix)]

for filename in tqdm(pgm_files):
    path = 'Test/' + filename
    with open (path, 'rb') as pgmf:
        image = plt.imread(pgmf)
        X_kag.append(image)
        
X = np.array([feature.hog(im) for im in tqdm(X_kag, desc='Construyendo X') ])


Construyendo X: 100%|██████████| 8000/8000 [00:03<00:00, 2326.72it/s]


In [10]:
from sklearn.decomposition import PCA 

pca = PCA(n_components=500)
X_pca_kag = pca.fit_transform(X)

# Predicción
y_kag = model.predict(X_pca_kag)

ValueError: X has 500 features, but KNeighborsClassifier is expecting 2916 features as input.

In [12]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [13]:
kaggle_hat.head()

,id,target_feature
2069,0,0
2527,1,0
3030,2,0
2576,3,0
4060,4,0


In [14]:
from datetime import datetime

fecha_hora = datetime.now().strftime("%Y-%m-%d %H:%M")

print(fecha_hora)

2025-06-11 19:39


In [15]:
# Guardamos en un .csv para subir a kaggle
submission_name = f'knn_{fecha_hora}'
kaggle_hat.to_csv(submission_name + '.csv', index=False)